In [ ]:
sc.getConf.toDebugString

 #  RDD API Examples

## Word Count
In this example, we use a few transformations to build a dataset of (String, Int) pairs called counts and then save it to a file.
```
sc.textFile(name, minPartitions=None, use_unicode=True)
Read a text file from HDFS, a local file system (available on all nodes), or any Hadoop-supported file system URI, and return it as an RDD of Strings.
```

In [ ]:
import scala.io.Source

//text_file = sc.textFile("/../datasets/quijote.txt")
// To avoid copying a local file to all workers


val textFile = sc.parallelize(Source.fromFile("../datasets/quijote.txt").getLines.toList)



val counts = textFile.flatMap(line => line.split(" "))
                 .map(word => (word, 1))
                 .reduceByKey(_ + _)
counts.sortBy(_._2, ascending=false).take(50)



 ## Pi Estimation

Spark can also be used for compute-intensive tasks. This code estimates pi by "throwing darts" at a circle. We pick random points in the unit square ((0, 0) to (1,1)) and see how many fall in the unit circle. The fraction should be pi / 4, so we use this to get our estimate.

In [ ]:
val NUM_SAMPLES = 1000000 
val count = sc.parallelize(1 to NUM_SAMPLES).filter { _ =>
  val x = math.random
  val y = math.random
  x*x + y*y < 1
}.count()
println(s"Pi is roughly ${4.0 * count / NUM_SAMPLES}")

# DataFrame API Examples

In this example, we count all quijote lines mentioning Dulcinea.

In [ ]:
import org.apache.spark.sql.functions.col
import scala.io.Source

//text_file = sc.textFile("/../datasets/quijote.txt")
// To avoid copying a local file to all workers
val textFile = sc.parallelize(Source.fromFile("../datasets/quijote.txt").getLines.toList)

// Creates a DataFrame having a single column named "line"
val df = textFile.toDF("line")
val dulcinea_lines = df.filter(col("line").like("%Dulcinea%"))
// Counts all the Dulcinea lines
printf("Lines with 'Dulcinea' = %d\n", dulcinea_lines.count())
// Counts lines mentioning Dulcinea and Quijote
val dulcinea_quijote_lines = dulcinea_lines.filter(col("line").like("%Quijote%"))
printf("Lines with 'Dulcinea' and 'Quijote' = %d\n", dulcinea_quijote_lines.count())

In [ ]:
%%dataframe
dulcinea_quijote_lines